In [1]:
import os
os.chdir("/Users/komangandikawirasantosa/Face-Rec/DeepFace-IPYNB")

In [2]:
import cv2
import numpy as np
import os
from insightface import app
from scipy.spatial.distance import cosine

# Load the InsightFace model
face_recognition = app.FaceAnalysis()
face_recognition.prepare(ctx_id=-1, det_size=(640, 640))

# Load the face database
database_path = 'Database'  # Replace with the path to your database
face_embeddings = {}
for person_name in os.listdir(database_path):
    person_folder = os.path.join(database_path, person_name)
    if os.path.isdir(person_folder):
        face_embeddings[person_name] = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                faces = face_recognition.get(img)
                for face in faces:
                    # Store the normalized embedding of the first face detected
                    face_embeddings[person_name].append(face.normed_embedding)


/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/komangandikawirasantosa/.in

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [ ]:
cap = cv2.VideoCapture(1)

# Set camera properties for better image quality
cap.set(cv2.CAP_PROP_BRIGHTNESS, 0.5)  # Adjust brightness (range: 0.0 - 1.0)
cap.set(cv2.CAP_PROP_CONTRAST, 0.5)    # Adjust contrast (range: 0.0 - 1.0)
cap.set(cv2.CAP_PROP_EXPOSURE, 0.1)     # Adjust exposure (may vary by camera)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    # Perform face detection and recognition
    faces = face_recognition.get(frame)

    # Display results
    for face in faces:
        # Get bounding box and landmarks
        bbox = face.bbox.astype(int)
        identity = face.normed_embedding

        # Compare with the database
        recognized_person = None
        min_distance = float('inf')
        for person_name, embeddings in face_embeddings.items():
            for embedding in embeddings:
                distance = cosine(identity, embedding)
                if distance < min_distance:
                    min_distance = distance
                    recognized_person = person_name

        # Draw bounding box and label on the image
        if recognized_person is not None and min_distance < 0.65:  # Adjust threshold as necessary
            label = recognized_person
        else:
            label = "Unknown"

        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        cv2.putText(frame, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the resulting frame
    cv2.imshow('Face Recognition', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()